<a href="https://colab.research.google.com/github/MuajiiTsai/110-1-NTU-DBME5028/blob/main_/DeepLearningMidTerm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from sklearn import metrics
from PIL import Image
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm
from google.colab import drive

model_path = "/content/drive/My Drive/Deep_Learning/Midterm/model.pt"

##Connect Google Drive

In [2]:
drive.mount('/content/drive')
train_path = '/content/drive/My Drive/Deep_Learning/Midterm/train/'
os.chdir(train_path)
training_data_file = os.listdir(train_path)
print(training_data_file[999])
type(training_data_file)
# os.chdir(test_path)
# testing_data_file = os.listdir(test_path)
# print(len(training_data_file))

Mounted at /content/drive
WRIST_f810f6dbf6_image3.png


list

##Label preprocessing
跑一次就好

##Dataset

In [9]:
from torchvision.io import read_image, ImageReadMode
#train_dataset

Resize = transforms.Compose([
  transforms.Resize([512,512])
])

class BoneDataset(Dataset):
    def __init__(self, annotations_file, directory, transform=None, target_transform=None):
        self.dir = directory
        self.img_labels = pd.read_csv(annotations_file, dtype={'id':str, 'label':np.float64})
        self.img_dir = os.listdir(directory)
        self.transform = transform
        self.target_transform = target_transform
        self.item = self.classification()
    def __len__(self):
        return len(self.item)
    def __getitem__(self, idx):
        # img_path = self.img_dir[idx]
        # image = read_image(os.path.join(self.dir, img_path), mode=ImageReadMode.RGB)
        # 
        # label = getlabel(img_path)
        img_path = self.item.iloc[idx, 0]
        image = read_image(os.path.join(self.dir, img_path), mode=ImageReadMode.RGB)
        image = image.type(torch.FloatTensor)
        label = self.item.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label
    def classification(self):
        '''
        img list -> img dataframe(pandas)
        pair the img and the label
        del the img labeled with "nan"
        '''
        img_with_labels = pd.DataFrame(columns=['img_list', 'label'])
        # add the row
        self.img_labels.set_index('id', inplace=True)
        for i in range(len(self.img_dir)):
            j = self.img_labels.loc[self.img_dir[i][:-11]][0]
            if not np.isnan(j):
                new = pd.DataFrame({
                    'img_list': [self.img_dir[i]],
                    'label': [j]
                })
                img_with_labels = img_with_labels.append(new, ignore_index=True)

        self.img_labels.reset_index(inplace=True)
        return img_with_labels   # The dataframe contained all labeled img

In [10]:
train_path = '/content/drive/My Drive/Deep_Learning/Midterm/train/'
test_path = '/content/drive/My Drive/Deep_Learning/Midterm/test/'
label_file = '/content/drive/My Drive/Deep_Learning/Midterm/train.csv'

#model
batch_size = 4
learning_rate = 1e-4
num_show = 3
transform = transforms.Compose([
  transforms.Grayscale(num_output_channels=3),
  # transforms.PILToTensor(),
  transforms.Resize([512,512]),
  # transforms.ToPILImage(mode='RGB')
])



train_dataset = BoneDataset(label_file, train_path, transform=transform)
# test_dataset = ...


train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle = True
)
'''
test_dataloader = torch.utils.data.DataLoader(
    #test_dataset,
    batch_size=1,
    shuffle = False
)
'''
model = models.efficientnet_b0(pretrained=True)
# model = models.resnet18(pretrained=True)
model.conv1 = nn.Conv2d(
    1,
    64,
    kernel_size=(7, 7),
    stride=(2, 2),
    padding=(3, 3),
    bias=False
)
model = nn.Sequential(model, nn.Linear(1000,1))
model = model.cuda()
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
loss_fn = nn.SmoothL1Loss()
# print(train_dataloader.dataset)

In [5]:
train_features, train_labels = next(iter(train_dataloader))
print(f"feature batch size: {train_features.size()}")
print(f"labels batch size: {train_labels.size()}")
img = train_features[1].squeeze()
label = train_labels[1]
# print(img)
# plt.imshow(img)
# plt.show()
print(f"Label: {label}")

feature batch size: torch.Size([4, 3, 512, 512])
labels batch size: torch.Size([4])
Label: 0.0


In [11]:
num_epochs = 1

#train the model
best_test_loss = np.inf

for idx_epoch in range(num_epochs):
  print(f"Epoch{idx_epoch}")

  #training phase
  model.train()

  train_losses = []
  for image_batch, label_batch in tqdm(train_dataloader):
    optimizer.zero_grad()
    image_batch = image_batch.cuda()
    label_batch = label_batch.cuda()
    output_batch = model(image_batch)
    # print(output_batch.shape)
    loss = loss_fn(output_batch, label_batch)
    loss.backward()
    optimizer.step()
    
    loss = loss.detach().cpu().numpy()
    train_losses.append(loss)

  train_loss = np.mean(train_losses)
  print(f"Training loss: {train_loss}")

  # y = np.array([1,1,2,2])
  # y_pred = np.array([0.1,0.4,0.35,2])
  # fpr, tpr, thresholds = metrics.roc_curve(y, y_pred, pos_label=2)
  # print(metrics.auc(fpr, tpr))

  '''
  #testing phase
  model.eval()

  test_losses = []
  test_accuracies = []
  for image_batch, label_batch in tqdm(test_dataloader):
    image_batch = image_batch.cuda()
    label_batch = label_batch.cuda()
    output_batch = model(image_batch)
    loss = loss_fn(output_batch, label_batch)
    
    loss = loss.detach().cpu().numpy()
    train_losses.append(loss)
    
    ##### AUC #####
    

    accuracy = np.mean(is_correct_batch)
    test_accuracies.append(accuracy)
    
    

    #####
    test_loss = np.mean(test_losses)
    test_accuracy = np.mean(test_accuracies)

    if test_loss < best_test_loss:
      best_test_loss = test_loss
      torch.save(model.state_dict(), model_path)
      print("The model is saved")
    '''
  torch.save(model.state_dict(), model_path)
  print("The model is saved")

Epoch0


  0%|          | 0/1768 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:912: UserWarning: Using a target size (torch.Size([4])) that is different to the input size (torch.Size([4, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.smooth_l1_loss(input, target, reduction=self.reduction, beta=self.beta)


Training loss: 0.12012789654461592
The model is saved


#Playground

##Import CSV

In [ ]:
df = pd.read_csv (r'/content/drive/My Drive/Deep_Learning/Midterm/train.csv')
# def a():
#     df = pd.read_csv (r'/content/drive/My Drive/Deep_Learning/Midterm/train.csv')
#     return df
# x = a()
# sorted_df = df.sort_values(by="label",kind='mergesort')
# i = df.loc["FOREARM_00f72d1153"]
df.set_index('id', inplace=True)
j = df.loc['FOREARM_000b20cbe3'][0]
print(type(j))
print(type("str"))
# j.reset_index(inplace=True)
df.reset_index(inplace=True)
# print(f"j0: {j[0]}\n")
# print(j['id'][0])
print(j)
print(df)

<class 'numpy.float64'>
<class 'str'>
nan
                      id  label
0     FOREARM_000b20cbe3    NaN
1     FOREARM_000be2e9ab    1.0
2     FOREARM_00192d1b21    NaN
3     FOREARM_0055c4c62a    1.0
4     FOREARM_00f72d1153    0.0
...                  ...    ...
3995    WRIST_ff39824aa4    0.0
3996    WRIST_ff5128afcd    1.0
3997    WRIST_ff60656c01    0.0
3998    WRIST_ffaf60a955    0.0
3999    WRIST_fff3b05bc0    0.0

[4000 rows x 2 columns]


##Import Data

In [ ]:
train_dataset = torch.tensor([])
transform1 = transforms.Compose([
  transforms.PILToTensor(),
])
print(train_dataset)
for i in range(100):
  img = Image.open(f"{train_path}/{training_data_file[i]}")
  img_tensor = transform1(img)
  a = torch.cat((train_dataset, img_tensor),0)
  # print(a.shape)

tensor([])


KeyboardInterrupt: ignored